In [ ]:
from protons.app.ligand_tautomers import *
from protons.app import logger
from protons.app.logger import log
from lxml import etree
import json
import sys
import rdkit
from protons import app
from simtk import unit, openmm as mm
from protons.app import ConstantPHCalibration, ForceFieldProtonDrive, NCMCProtonDrive
from protons.app import MetadataReporter, TitrationReporter, NCMCReporter, SAMSReporter
import shutil
import signal
import json
from protons.app.logger import log, logging
import numpy as np
from openmmtools.integrators import LangevinIntegrator, ExternalPerturbationLangevinIntegrator
log.setLevel(logging.DEBUG)
import os
import sys
import netCDF4
from saltswap.wrappers import Salinator
import json


In [ ]:
def setting_up_tautomer():
    # Retrieve parameter fields

    pH = 7.8
    max_penalty = 10
    tautomerize = True
    resname = 'UNL'

    ifolder = '/home/mwieder/Work/Projects/tautomers-protons//protons/UNL-testdata/input/'
    ofolder = '/home/mwieder/Work/Projects/tautomers-protons//protons/UNL-testdata/output/'

    #retrieve input fields
    icalib = ifolder + 'tautomer_set1_mol1.pdb'
    input_mol2 = ifolder + 'tautomer_set1_mol1.mol2'
    hydrogen_def = ofolder + 'tautomer_set.hxml'

    offxml = ofolder + 'tautomer_set.ffxml'
    ocalib = ofolder + 'tautomer_set.cif'


    # Debugging/intermediate files
    dhydrogen_fix = ofolder + 'tautomer_set1_hydrogen_fixed.mol2'
    dkeep_files = True

    

    if not os.path.isdir(ofolder):
        os.makedirs(ofolder)

    # Begin processing

    # process into mol2
    prepare_mol2_for_parametrization(input_mol2, dhydrogen_fix)
    # retrieve states


    # run epik
    t1 =  { 'log_population' : '7.1', 'net_charge' : 0 }
    t2 =  { 'log_population' : '1', 'net_charge' : 0 }
    isomer_info = [ t1, t2 ]



    # parametrize
    s, compiler = generate_protons_ffxml(dhydrogen_fix, isomer_info, offxml, pH, resname=resname)
    # create hydrogens
    create_hydrogen_definitions(offxml, hydrogen_def)
    print('Finished with creating hydrogen definitions!')

    # set up calibration system
    #extract_residue('/home/mwieder/Work/Projects/protons/mw/example_scripts/ligand-parameter/t12.pdb',dext_res,resname='UNL')

    # prepare solvated system
    sim = prepare_calibration_system(icalib, ocalib, offxml, hydrogen_def)
    return compiler


In [ ]:
compiler = setting_up_tautomer()

In [ ]:
# Define what to do on timeout
class TimeOutError(RuntimeError):
    """This error is raised when an operation is taking longer than expected."""
    pass


def timeout_handler(signum, frame):
    """Handle a timeout."""
    log.warn("Script is running out of time. Attempting to exit cleanly.")
    raise TimeOutError("Running out of time, shutting down!")

def serialize_state(context, outputfile):
    """
    Serialize the simulation state to xml.
    """
    xmls = mm.XmlSerializer
    statexml = xmls.serialize(context.getState(getPositions=True, getVelocities=True))
    with open(outputfile,'w') as statefile:
        statefile.write(statexml)

def serialize_drive(drive, outputfile):
    """
    Serialize the drive residues to xml.
    """
    drivexml = drive.serialize_titration_groups()
    with open(outputfile, 'wb') as drivefile:
        drivefile.write(drivexml)

def serialize_sams_status(calibration, outputfile):
    """
    Serialize the state of the SAMS calibration as json
    """
    samsProperties = calibration.export_samsProperties()
    samsjson = json.dumps(samsProperties, sort_keys=True, indent=4, separators=(',', ': '))
    with open(outputfile, 'w') as samsfile:
        samsfile.write(samsjson)

class ExternalGBAOABIntegrator(ExternalPerturbationLangevinIntegrator):
    """
    Implementation of the gBAOAB integrator which tracks external protocol work.

    Parameters
    ----------
        number_R: int, default: 1
            The number of sequential R steps.  For instance V R R O R R V has number_R = 2
        temperature : simtk.unit.Quantity compatible with kelvin, default: 298*unit.kelvin
           The temperature.
        collision_rate : simtk.unit.Quantity compatible with 1/picoseconds, default: 1.0/unit.picoseconds
           The collision rate.
        timestep : simtk.unit.Quantity compatible with femtoseconds, default: 1.0*unit.femtoseconds
           The integration timestep.


    """

    def __init__(self, number_R_steps=1, temperature=298.0 * unit.kelvin,
                 collision_rate=1.0 / unit.picoseconds,
                 timestep=1.0 * unit.femtoseconds,
                 constraint_tolerance=1e-7
                 ):
        Rstep = " R" * number_R_steps

        super(ExternalGBAOABIntegrator, self).__init__(splitting="V{0} O{0} V".format(Rstep),
                                                       temperature=temperature,
                                                       collision_rate=collision_rate,
                                                       timestep=timestep,
                                                       constraint_tolerance=constraint_tolerance,
                                                       measure_shadow_work=False,
                                                       measure_heat=False,
                                                       )


def calibrate_system(compiler):
    """Main simulation loop."""
    ofolder = '/home/mwieder/Work/Projects/tautomers-protons/protons/UNL-testdata/output/'

    input_pdbx_file = ofolder + '/tautomer_set.cif'
    custom_xml = ofolder + '/tautomer_set.ffxml'

    forcefield = app.ForceField('amber10-constph.xml', 'gaff.xml', custom_xml, 'tip3p.xml', 'ions_tip3p.xml')

    # Load the PDBxfile and the forcefield files
    pdb_object = app.PDBxFile(input_pdbx_file)

    # Prepare the Simulation
    topology = pdb_object.topology
    
    
    positions = pdb_object.positions

    # Quick fix for histidines in topology
    # Openmm relabels them HIS, which leads to them not being detected as
    # titratable. Renaming them fixes this.
    for residue in topology.residues():
        if residue.name == 'HIS':
            residue.name = 'HIP'

    name_netcdf = "test.netcdf"
    dcd_output_name = "test.dcd"


    output_context_xml = "state.xml"
    output_drive_xml = "drive.xml"
    output_calibration_json = "calibration.json"

    # Integrator options
    timestep = 2.0 * unit.femtosecond
    constraint_tolerance = 1e-07
    collision_rate = 1.0 / unit.picosecond
    number_R_steps = 1

    # Steps of MD before starting the main loop
    num_thermalization_steps = 10000
    # Steps of MD in between MC moves
    steps_between_updates = 10000

    counterion_method = 'chenroux'
    ncmc_steps_per_trial = 10000
    prop_steps_per_trial = 1
    total_iterations = 10000

    modulo_ligand_update = 1 # Update ligand every n iterations

    # settings form minimization
    pre_run_minimization_tolerance = float(1e-05) * unit.kilojoule / unit.mole
    minimization_max_iterations = 0

    # SAMS settings
    sams = {}
    beta_sams = 0.5
    flatness_criterion = 0.15
    sams[beta_sams] = beta_sams 
    sams[flatness_criterion] = flatness_criterion

    # Script specific settings
    script_timeout = 428400 # 119 hours

    # Platform Options

    platform = mm.Platform.getPlatformByName('CUDA')
    #platform = mm.Platform.getPlatformByName('CPU')
    properties = {'CudaPrecision': 'mixed', 'DeterministicForces': 'true'}
    # System Configuration
    nonbondedMethod = app.PME
    constraints = app.HBonds
    rigidWater = True
    ewaldErrorTolerance = 1e-05
    barostatInterval =  25
    switching_distance = 0.85 * unit.nanometers
    nonbondedCutoff = 1.0 * unit.nanometers
    pressure = 1.0 * unit.atmosphere
    temperature = 300.0 * unit.kelvin

    system = forcefield.createSystem(topology, nonbondedMethod=nonbondedMethod, constraints=constraints,
                                     rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance, nonbondedCutoff=nonbondedCutoff)

    #
    for force in system.getForces():
        if isinstance(force, mm.NonbondedForce):
            force.setUseSwitchingFunction(True)

            force.setSwitchingDistance(switching_distance)

    # NPT simulation
    system.addForce(
        mm.MonteCarloBarostat(
            pressure,
            temperature,
            barostatInterval))


    integrator = ExternalGBAOABIntegrator(number_R_steps=number_R_steps, temperature=temperature, collision_rate=collision_rate, timestep=timestep, constraint_tolerance=constraint_tolerance)
    ncmc_propagation_integrator = ExternalGBAOABIntegrator(number_R_steps=number_R_steps, temperature=temperature, collision_rate=collision_rate, timestep=timestep, constraint_tolerance=constraint_tolerance)

    # Define a compound integrator
    compound_integrator = mm.CompoundIntegrator()
    compound_integrator.addIntegrator(integrator)
    compound_integrator.addIntegrator(ncmc_propagation_integrator)
    compound_integrator.setCurrentIntegrator(0)

    driver = ForceFieldProtonDrive(temperature, topology, system, forcefield, ['amber10-constph.xml', custom_xml], compiler,pressure=pressure,
                                           perturbations_per_trial=ncmc_steps_per_trial, propagations_per_step=prop_steps_per_trial, residues_by_name=['UNL'])
    
    # Assumes ligand is always the last titration group
    ligand_titration_group_index = len(driver.titrationGroups) - 1
    # Define residue pools
    pools = {'ligand' : [ligand_titration_group_index]}
    driver.define_pools(pools)
    # Create SAMS sampler
    print('Setting up Constant pH Calibartion ...')
    simulation = app.ConstantPHCalibration(topology, system, compound_integrator, driver, group_index=ligand_titration_group_index, platform=platform, samsProperties=sams)
    simulation.context.setPositions(positions)
    salinator = Salinator(context=simulation.context,
                            system=system,
                            topology=topology,
                            ncmc_integrator=compound_integrator.getIntegrator(1),
                            salt_concentration=0.150 * unit.molar,
                            pressure=pressure,
                            temperature=temperature)
    salinator.neutralize()
    
    salinator.initialize_concentration()
    swapper = salinator.swapper
    # if chen-roux required, attach swapper. Else, use neutralizing background charge
    if counterion_method in ["chenroux", "chen-roux"]:
        simulation.drive.attach_swapper(swapper)

    simulation.minimizeEnergy(tolerance=pre_run_minimization_tolerance, maxIterations=minimization_max_iterations)
    simulation.step(num_thermalization_steps)
    print('Starting with simulation')

    dcdreporter = app.DCDReporter(dcd_output_name, int(steps_between_updates/10))
    ncfile = netCDF4.Dataset(name_netcdf, "w")
    metdatarep = MetadataReporter(ncfile)
    ncmcrep = NCMCReporter(ncfile,1)
    titrep = TitrationReporter(ncfile,1)
    simulation.reporters.append(dcdreporter)
    simulation.update_reporters.append(metdatarep)
    simulation.update_reporters.append(ncmcrep)
    simulation.update_reporters.append(titrep)
    samsrep = SAMSReporter(ncfile,1)
    simulation.calibration_reporters.append(samsrep)
    pdb = mm.app.PDBFile('/home/mwieder/input.pdb')
    
    # Raises an exception if the simulation runs out of time, so that the script can be killed cleanly from within python
    signal.alarm(script_timeout)
    
    try:
        for i in range(total_iterations):
            log.info("Iteration %i", i)
            if i == 5:
                log.info("Simulation seems to be working. Suppressing debugging info.")
                log.setLevel(logging.INFO)
            pos = simulation.context.getState(getPositions=True).getPositions() 
            mm.app.PDBFile.writeFile(pdb.topology, pos, open('/home/mwieder/tmp_wd/00_before_it_'+str(i)+'_test.pdb', 'w'))
            simulation.step(steps_between_updates)
            pos = simulation.context.getState(getPositions=True).getPositions() 
            mm.app.PDBFile.writeFile(pdb.topology, pos, open('/home/mwieder/tmp_wd/00_after_it_'+str(i)+'_test.pdb', 'w'))
            simulation.update(1, pool='ligand')
            simulation.adapt()
        # Reset timer
        signal.alarm(0)

    except TimeOutError:
        log.warn("Simulation ran out of time, saving current results.")

    finally:
        # export the context
        serialize_state(simulation.context,output_context_xml)
        # export the driver
        serialize_drive(simulation.drive, output_drive_xml)
        # export the calibration status
        serialize_sams_status(simulation, output_calibration_json)

        ncfile.close()

    return topology

    # End of script

In [ ]:
t = calibrate_system(compiler)

In [ ]:
% cat /home/mwieder/Work/Projects/tautomers/protons/UNL-testdata/output/tautomer_set1.hxml

In [ ]:
% cat dummy.xml

In [ ]:
% less ../../../tautomers-protons/protons/UNL-testdata/output/tautomer_set.ffxml

In [ ]:
% cat ../../../tautomers/protons/UNL-testdata/output/tautomer_set1.ffxml 

In [ ]:
% cat  ~/test.xml

In [ ]:
% cat tautomer_pair_mol1.hxml

In [ ]:
% cat tautomer_pair_mol1_fixed_hydrogens.mol2